# Recommendation system 

Only suggests similar titles based on the genre similarity, Score, Popularity and plot summary

## Imports

In [1]:
import pandas as pd
import numpy as np
import pickle 

import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
lm = WordNetLemmatizer()
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import nltk                                         #Natural language processing tool-kit

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer
stop_words = set(stopwords.words('english')) 

import gc
import math

import yake

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler

## Loading dataset

Download from [kaggle link](https://www.kaggle.com/snehaanbhawal/netflix-tv-shows-and-movie-list)

Dropping all the colums except only the Genre and Title

In [2]:
df = pd.read_csv("netflix_list.csv")

In [3]:
df.drop(["popular_rank", "type", "episodes", "genres", "orign_country", "endYear","certificate","isAdult","summary","cast","plot"], axis=1, inplace=True)
# df.dropna(inplace = True)
df.reset_index(drop=True, inplace=True)

In [4]:
df1 = pd.read_csv("tag.csv")
df = pd.merge(df, df1, on='title')

df.drop(['cast'], axis=1, inplace=True)

In [5]:
df.isnull().sum()

imdb_id        0
title          0
startYear    451
runtime        0
language       0
rating       771
numVotes     771
image_url      0
story          0
summary        0
genres         0
tags           0
dtype: int64

In [6]:
df.head(1)

,imdb_id,title,startYear,runtime,language,rating,numVotes,image_url,story,summary,genres,tags
0,tt4052886,Lucifer,2016.0,42,English,8.1,250884.0,https://m.media-amazon.com/images/M/MV5BNzY1Yj...,Lucifer Morningstar has decided he's had enoug...,"Lucifer Morningstar, bored from his sulking li...","Crime,Drama,Fantasy","['Morningstar', 'Angeles', 'Los', 'Lucifer', '..."
1,tt0993840,Army of the Dead,2021.0,148,English,5.8,110780.0,https://m.media-amazon.com/images/M/MV5BNGY0Nz...,"Following a zombie outbreak in Las Vegas, a gr...","With the abandoned, walled city of Las Vegas o...","Action,Crime,Horror","['city', 'Bly', 'Las', 'Sin', 'Tanaka', 'milli..."
2,tt7255502,The Kominsky Method,2018.0,30,English,8.2,28795.0,https://m.media-amazon.com/images/M/MV5BMzA0YT...,"An aging actor, who long ago enjoyed a brush w...",Michael Douglas plays an actor who made it big...,"Comedy,Drama","['Method', 'Douglas', 'Kominsky', 'Michael', '..."
3,tt0108778,Friends,1994.0,22,English,8.9,861843.0,https://m.media-amazon.com/images/M/MV5BNDVkYj...,Follows the personal and professional lives of...,"Ross Geller, Rachel Green, Monica Geller, Joey...","Comedy,Romance","['Geller', 'Rachel', 'Green', 'Monica', 'Joey'..."
4,tt9251798,Ragnarok,2020.0,45,Norwegian,7.5,26606.0,https://m.media-amazon.com/images/M/MV5BODM3NT...,A small Norwegian town experiencing warm winte...,In the small fictional town of Edda coming of ...,"Action,Drama,Fantasy","['Edda', 'people', 'small', 'fictional', 'town..."


In [7]:
df.shape

(6608, 12)

In [8]:
for i in range(df.shape[0]):
    if df.title[i][0] == " ":
        df.title[i] = df.title[i][1:]
        
df.drop_duplicates(subset='title', keep='first', inplace=True)
df.reset_index(inplace = True, drop=True)

<ipython-input-8-c664b2c8074e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.title[i] = df.title[i][1:]


In [9]:
df.shape

(6037, 12)

## Some Processing data

In [10]:
df.tags = [str(x).replace(","," ").replace("["," ").replace("]"," ").replace("'"," ") for x in df.tags]

df = df[df.runtime != "\\N" ]
df = df[df.startYear != "\\N" ]
df = df[df.genres != "\\N" ]



df.rating = [x for x in df.rating]
df = df[df.rating > 5 ]

df = df[df.numVotes >= 500]
df.numVotes = [int(x/5000) for x in df.numVotes]

df.startYear = [int(x) for x in df.startYear]
df = df[df.startYear >= 1990 ]
df.startYear = [math.ceil((x%1990)/5) for x in df.startYear]

df.runtime = [int(x) for x in df.runtime]
df = df[df.runtime >= 10 ]

In [11]:
df.isnull().sum()

imdb_id      0
title        0
startYear    0
runtime      0
language     0
rating       0
numVotes     0
image_url    0
story        0
summary      0
genres       0
tags         0
dtype: int64

In [12]:
df.reset_index(inplace=True, drop=True)

In [14]:
for i in range(df.shape[0]):
    if df.runtime[i]<=30:
        df.runtime[i] = "short_runtime"
    elif df.runtime[i]>30 and df.runtime[i]<=60:
        df.runtime[i] = "normal_runtime"
    elif df.runtime[i]>60:   
        df.runtime[i]="long_runtime"

C:\Users\mailt\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-14-21f3295af26a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.runtime[i]="long_runtime"
<ipython-input-14-21f3295af26a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.runtime[i] = "short_runtime"
<ipython-input-14-21f3295af26a>:5: SettingWithCo

In [15]:
for i in range(df.shape[0]):
    if (df.language[i] =="English"):# or (df.language[i]!="Spanish") or (df.language[i]!="Korean") or (df.language[i]!="Hindi") or (df.language[i]!="French"):
        pass
    elif (df.language[i] =="Spanish"):# or (df.language[i]!="Spanish") or (df.language[i]!="Korean") or (df.language[i]!="Hindi") or (df.language[i]!="French"):
        pass
    elif (df.language[i] =="Korean"):# or (df.language[i]!="Spanish") or (df.language[i]!="Korean") or (df.language[i]!="Hindi") or (df.language[i]!="French"):
        pass
    elif (df.language[i] =="Hindi"):# or (df.language[i]!="Spanish") or (df.language[i]!="Korean") or (df.language[i]!="Hindi") or (df.language[i]!="French"):
        pass
    elif (df.language[i] =="French"):# or (df.language[i]!="Spanish") or (df.language[i]!="Korean") or (df.language[i]!="Hindi") or (df.language[i]!="French"):
        pass
    else:
        df.language[i] = "Other"
    

<ipython-input-15-0b7278430f6b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.language[i] = "Other"


In [17]:
df.head(1)

,imdb_id,title,startYear,runtime,language,rating,numVotes,image_url,story,summary,genres,tags
0,tt4052886,Lucifer,6,normal_runtime,English,8.1,50,https://m.media-amazon.com/images/M/MV5BNzY1Yj...,Lucifer Morningstar has decided he's had enoug...,"Lucifer Morningstar, bored from his sulking li...","Crime,Drama,Fantasy",Morningstar Angeles Los Lucifer ...


In [18]:
copy = df[['imdb_id', 'title', 'rating', 'image_url', 'story', 'summary','genres']]

In [20]:
v = CountVectorizer()
x = v.fit_transform(df["runtime"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df = pd.concat([df, df1], axis=1)
df.drop(["runtime"], axis = 1, inplace=True) 

In [21]:
v = CountVectorizer()
x = v.fit_transform(df["language"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df = pd.concat([df, df1], axis=1)
df.drop(["language"], axis = 1, inplace=True) 

In [22]:
scaler = MinMaxScaler()
df[['startYear', 'rating', 'numVotes']] = scaler.fit_transform(df[['startYear', 'rating', 'numVotes']])

In [23]:
v = TfidfVectorizer()
x = v.fit_transform(df["genres"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df = pd.concat([df, df1], axis=1)
# df.drop(["genres"], axis = 1, inplace=True) 

In [24]:
v = TfidfVectorizer()
x = v.fit_transform(df["tags"])
df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df1 = pd.concat([df, df1], axis=1)
df1.drop(["tags",'imdb_id', 'title', 'image_url', 'story', 'summary','genres'], axis = 1, inplace=True) 

In [25]:
df1.head(2)

,startYear,rating,numVotes,long_runtime,normal_runtime,short_runtime,english,french,hindi,korean,...,álvaro,ángel,ángela,åland,åsa,çurgunlu,état,ömer,örjan,úrsula
0,0.857143,0.697674,0.147493,0,1,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.162791,0.064897,1,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
import scipy
mat = scipy.sparse.csr_matrix(df1.values)
from sklearn.metrics.pairwise import sigmoid_kernel
sig1 = sigmoid_kernel(mat,mat)

In [27]:
np.save('sugg_data.npy', sig1)

In [28]:
sig1 = np.load('sugg_data.npy')

In [29]:
import pickle
copy.to_pickle("title_list.pkl")

In [30]:
df = pd.read_pickle("title_list.pkl")

In [31]:
df.head()

,imdb_id,title,rating,image_url,story,summary,genres
0,tt4052886,Lucifer,8.1,https://m.media-amazon.com/images/M/MV5BNzY1Yj...,Lucifer Morningstar has decided he's had enoug...,"Lucifer Morningstar, bored from his sulking li...","Crime,Drama,Fantasy"
1,tt0993840,Army of the Dead,5.8,https://m.media-amazon.com/images/M/MV5BNGY0Nz...,"Following a zombie outbreak in Las Vegas, a gr...","With the abandoned, walled city of Las Vegas o...","Action,Crime,Horror"
2,tt7255502,The Kominsky Method,8.2,https://m.media-amazon.com/images/M/MV5BMzA0YT...,"An aging actor, who long ago enjoyed a brush w...",Michael Douglas plays an actor who made it big...,"Comedy,Drama"
3,tt0108778,Friends,8.9,https://m.media-amazon.com/images/M/MV5BNDVkYj...,Follows the personal and professional lives of...,"Ross Geller, Rachel Green, Monica Geller, Joey...","Comedy,Romance"
4,tt9251798,Ragnarok,7.5,https://m.media-amazon.com/images/M/MV5BODM3NT...,A small Norwegian town experiencing warm winte...,In the small fictional town of Edda coming of ...,"Action,Drama,Fantasy"


In [32]:
index = pd.Series(df.index, index = df.title).drop_duplicates()

In [33]:
def recommend(title, sig=sig1):
    idx = index[title]
    scores = list(enumerate(sig1[idx]))
    scores = sorted(scores, key=lambda x:x[1], reverse=True)
    scores= scores[1:10]
    mov_index = [i[0] for i in scores]
    return df.iloc[mov_index]['imdb_id']

In [34]:
recommend("Sacred Games")

1524    tt11905720
1172     tt8069036
131      tt5675620
2098    tt11725706
2133     tt5606154
1755     tt8105958
2306    tt10655506
137      tt3322312
2046     tt9359294
Name: imdb_id, dtype: object

In [65]:
index = df[df['imdb_id']=='tt10655506'].index.values

In [68]:
df.at[int(index),'title']

'Masaba Masaba'

In [69]:
def return_details(imdbID):
    details = {}
    index = df[df['imdb_id']==imdbID].index.values
    details['image_url'] = df.at[int(index),'image_url']
    details['title'] = df.at[int(index),'title']
    try:
    	try:
    		details['summary'] = df.at[int(index),'summary']
    	except:
    		details['summary'] = df.at[int(index),'story']
    except:
    	details['summary'] = "Unfortunately no Summary Found !"
    details['rating'] = df.at[int(index),'rating']
    details['genres'] = df.at[int(index),'genres']       
    return details   
  
#     querystring = {"tconst": imdbID}
#     response = requests.request("GET", url, headers=headers, params=querystring)
#     json_data = response.json()
#     details = {}
#     details['image_url'] = json_data['title']['image']['url']
#     details['title'] = json_data['title']['title']
#     try:
#     	try:
#     		details['summary'] = json_data['plotOutline']['text']
#     	except:
#     		details['summary'] = json_data['plotSummary']['text']
#     except:
#     	details['summary'] = "Unfortunately no Summary Found !"

#     details['rating'] = json_data['ratings']['rating']
#     details['genres'] = str(json_data['genres']).replace("[","").replace("]","").replace("'","")
#     return details  

In [70]:
print(return_details('tt11905720'))

{'image_url': 'https://m.media-amazon.com/images/M/MV5BYzc4ZmFhZjItOWJjMy00YzhiLTkxMDAtNWUyNjIyOTJkYmY3XkEyXkFqcGdeQXVyNDAzNDk0MTQ@._V1_FMjpg_UX1000_.jpg', 'title': 'She', 'summary': "An undercover assignment to expose a drug ring becomes a timid Mumbai constable's road to empowerment as she realizes her dormant sexuality's potential.", 'rating': 6.5, 'genres': 'Crime,Drama'}
